# Iceberg Classification Step 2a: Model Training with Single GPU
The following code includes demonstration for:
- get data from ``feature store``
- training with ``TFRecord`` on a single GPU
- hyperparameter optimization with ``maggy``

In [1]:
import tensorflow as tf
print("Version of TensorFlow is {}".format(tf.__version__))

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
51,application_1574692443370_0052,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.
Version of TensorFlow is 1.14.0

In [2]:
from hops import featurestore
from hops import tensorboard
from maggy import experiment
from maggy.callbacks import KerasBatchEnd
from maggy.callbacks import KerasEpochEnd

You are running maggy on Hopsworks.

In [3]:
def create_tf_dataset(tfrecord_path, name_list):
    dataset_dir = featurestore.get_training_dataset_path(tfrecord_path)
    input_files = tf.gfile.Glob(dataset_dir + "/part-r-*")
    dataset = tf.data.TFRecordDataset(input_files)
    # 'tf_record_schema' is needed because we need to parse a single example from all the TFRecords we have
    tf_record_schema = featurestore.get_training_dataset_tf_record_schema(tfrecord_path)

    def decode(example_proto):
        example = tf.parse_single_example(example_proto, tf_record_schema)
        x = tf.stack([example[name_list[0]], example[name_list[1]], example[name_list[2]]], axis=1)
        x = tf.reshape(x, [75, 75, 3])
        y = [tf.cast(example[name_list[3]], tf.float32)]
        return x,y
    
    dataset = dataset.map(decode).shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE).repeat(NUM_EPOCHS)
    return dataset

In [4]:
def create_model(kernel, pool, dropout):
    model = tf.keras.models.Sequential()
    
    #Conv Layer 1
    model.add(tf.keras.layers.Conv2D(64, kernel_size=(kernel, kernel), activation='relu', input_shape=INPUT_SHAPE))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(pool, pool), strides=(2,2)))
    model.add(tf.keras.layers.Dropout(dropout))

    #Conv Layer 2
    model.add(tf.keras.layers.Conv2D(128, kernel_size=(kernel, kernel), activation='relu' ))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(pool, pool), strides=(2, 2)))
    model.add(tf.keras.layers.Dropout(dropout))
    
    #Conv Layer 3
    model.add(tf.keras.layers.Conv2D(128, kernel_size=(3, 3), activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(tf.keras.layers.Dropout(0.2))

    #Conv Layer 4
    model.add(tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(tf.keras.layers.Dropout(0.2))

    #Flatten the data for upcoming dense layers
    model.add(tf.keras.layers.Flatten())

    #Dense Layers
    model.add(tf.keras.layers.Dense(512))
    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.Dropout(0.2))

    #Dense Layer 2
    model.add(tf.keras.layers.Dense(256))
    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.Dropout(0.2))

    #Sigmoid Layer
    model.add(tf.keras.layers.Dense(1))
    model.add(tf.keras.layers.Activation('sigmoid'))
    return model

In [18]:
#########
### maggy: hyperparameters as arguments and including the reporter
#########
def train_fn(kernel, pool, dropout, reporter):
    train_tfrecord_path = "train_tfrecords_iceberg_classification_dataset"
    train_name_list = ["band_1", "band_2", "band_avg", "is_iceberg"]
    train_dataset = create_tf_dataset(train_tfrecord_path, train_name_list)
    
    test_tfrecord_path = "test_tfrecords_iceberg_classification_dataset"
    test_name_list = ["band_1", "band_2", "band_avg", "is_iceberg"]
    test_dataset = create_tf_dataset(test_tfrecord_path, test_name_list)
    
    
    model = create_model(kernel, pool, dropout)
    model.compile(optimizer=tf.train.AdamOptimizer(LEARNING_RATE), loss='binary_crossentropy',  metrics=['accuracy'])

    #########
    ### maggy: REPORTER API through keras callback
    #########

    callbacks = [KerasBatchEnd(reporter, metric='acc')] # sina

    
    # fit the model
    history = model.fit(train_dataset, epochs=NUM_EPOCHS, steps_per_epoch=5, callbacks=callbacks)
    
    
    test_score = model.evaluate(test_dataset, verbose=1) # sina
    print('Test loss:', test_score[0]) # sina
    print('Test accuracy:', test_score[1]) # sina
    #########
    ### maggy: return the metric to be optimized, test accuracy in this case
    #########
    
    return test_score[1]

In [19]:
# Hyperparameter for TFRecords
NUM_EPOCHS = 2 # sina
BATCH_SIZE = 32
SHUFFLE_BUFFER_SIZE = 10000
# Hyperparameter for learning rate
LEARNING_RATE = 0.001
# Input shape of the model
INPUT_SHAPE= (75, 75, 3)

## Define the search space for hyperparameter optimization

In [20]:
from maggy import Searchspace

# The searchspace can be instantiated with parameters
sp = Searchspace()

# Or additional parameters can be added one by one
sp.add('kernel', ('INTEGER', [3, 4]))
sp.add('pool', ('INTEGER', [2, 3]))
sp.add('dropout', ('DOUBLE', [0.10, 0.50]))

Hyperparameter added: kernel
Hyperparameter added: pool
Hyperparameter added: dropout

## Launch the hyperparameter optimization

In [22]:
result = experiment.lagom(map_fun=train_fn, 
                           searchspace=sp, 
                           optimizer='randomsearch', 
                           direction='max', 
                           num_trials=10, 
                           name='Iceberg_Classification_Maggy', 
                           hb_interval=5,
                           es_interval=300,
                           es_min=5
                          )

0: Epoch 1/2
0: Epoch 2/2
0: Test loss: 0.6870104521512985
0: Test accuracy: 0.5375
0: Epoch 1/2
0: Epoch 2/2
0: Test loss: 0.6976054519414902
0: Test accuracy: 0.4625
0: Epoch 1/2
0: Epoch 2/2
0: Test loss: 0.6794597119092941
0: Test accuracy: 0.5375
0: Epoch 1/2
0: Epoch 2/2
0: Test loss: 0.6886839509010315
0: Test accuracy: 0.528125
0: Epoch 1/2
0: Epoch 2/2
0: Test loss: 0.649113941192627
0: Test accuracy: 0.66875
0: Epoch 1/2
0: Epoch 2/2
0: Test loss: 0.6886781752109528
0: Test accuracy: 0.50625
0: Epoch 1/2
0: Epoch 2/2
0: Test loss: 0.6886705309152603
0: Test accuracy: 0.5375
0: Epoch 1/2
0: Epoch 2/2
0: Test loss: 0.7179129153490067
0: Test accuracy: 0.4625
0: Epoch 1/2
0: Epoch 2/2
0: Test loss: 0.6826905578374862
0: Test accuracy: 0.5375
0: Epoch 1/2
0: Epoch 2/2
0: Test loss: 0.6882031589746476
0: Test accuracy: 0.61875

------ RandomSearch Results ------ direction(max) 
BEST combination {"kernel": 4, "pool": 3, "dropout": 0.2209323739211756} -- metric 0.66875
WORST combina

# End of Step2a